In [4]:
!pip install pyspark

In [5]:
 # iniciar a sessao
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder \
    .appName('videoAnalysis') \
    .getOrCreate()

In [8]:
# lendo o arquivo parquet
df_video = spark.read.parquet('/Users/Yuri/Desktop/ARQUIVOS PARQUET/videos-preparados.snappy.parquet')

In [9]:
# Mostrando informacoes Basicas (testando!)
print('Schema do DataFrame:')
df_video.printSchema()

Schema do DataFrame:
root
 |-- Title: string (nullable = true)
 |-- Video ID: string (nullable = true)
 |-- Published At: date (nullable = true)
 |-- Keyword: string (nullable = true)
 |-- Likes: integer (nullable = true)
 |-- Comments: integer (nullable = true)
 |-- Views: integer (nullable = true)
 |-- Interaction: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Keyword Index: double (nullable = true)
 |-- Features PCA: vector (nullable = true)
 |-- Features Normal: vector (nullable = true)
 |-- Features: vector (nullable = true)



In [10]:
# calcular quantidade de registros por keyword
contagem_keywords = df_video.groupBy('Keyword').count()

In [11]:
# ordernando por contagem e mostrando resultados
contagem_keywords.orderBy('count', ascending=False).show()

+----------------+-----+
|         Keyword|count|
+----------------+-----+
|             cnn|   50|
|       interview|   50|
|          crypto|   50|
|    data science|   50|
|        trolling|   50|
|        tutorial|   50|
|          marvel|   50|
|game development|   50|
|         mrbeast|   50|
|         physics|   50|
|             sat|   49|
|         history|   49|
|           cubes|   49|
|        reaction|   49|
|          sports|   49|
|            asmr|   49|
|computer science|   48|
|            food|   48|
|          how-to|   48|
|machine learning|   48|
+----------------+-----+
only showing top 20 rows


In [12]:
# mostrando todos resultados sem truncate(testando)
contagem_keywords.orderBy('count', ascending=False).show(truncate=False)

+----------------+-----+
|Keyword         |count|
+----------------+-----+
|cnn             |50   |
|interview       |50   |
|crypto          |50   |
|data science    |50   |
|trolling        |50   |
|tutorial        |50   |
|marvel          |50   |
|game development|50   |
|mrbeast         |50   |
|physics         |50   |
|sat             |49   |
|history         |49   |
|cubes           |49   |
|reaction        |49   |
|sports          |49   |
|asmr            |49   |
|computer science|48   |
|food            |48   |
|how-to          |48   |
|machine learning|48   |
+----------------+-----+
only showing top 20 rows


In [21]:
# calculando a media de interaction por keyworld
media_interaction_keyword = df_video.groupBy('Keyword').agg(
    {'Interaction': 'avg'}
)

In [23]:
# mostrando a media dos resultados
media_interaction_keyword.orderBy('avg(interaction)', ascending=False).show()

+-------------+--------------------+
|      Keyword|    avg(Interaction)|
+-------------+--------------------+
|      animals|  9.55066085263158E7|
|      mrbeast|       6.896586282E7|
|          bed|       5.438209175E7|
|        music|2.9691370304347824E7|
|      history| 1.565269257142857E7|
|        cubes|1.5043961224489795E7|
|      mukbang|1.1053630377777778E7|
|        apple|1.0873628214285715E7|
|       sports|   8695551.632653061|
|       how-to|           7975134.5|
|     business|   7310180.020833333|
|     tutorial|           6936688.3|
|       marvel|          6834159.44|
|         food|   5352944.104166667|
|       movies|   4897436.318181818|
|      biology|   4192382.063829787|
|         lofi|         4167085.875|
|      physics|          3795529.38|
|mathchemistry|  3427342.7333333334|
|    interview|          3044867.04|
+-------------+--------------------+
only showing top 20 rows


In [57]:
from pyspark.sql.functions import min, max, count, avg, variance, round, sum

rank_interactions_completo = df_video.groupBy('Keyword').agg(
    max('Interaction').alias('rank_interactions'),
    count('interaction').alias('total_registros'),
    avg('interaction').alias('media_interaction')
).orderBy('rank_interactions', ascending=False)

rank_interactions_completo.show()

+--------+-----------------+---------------+--------------------+
| Keyword|rank_interactions|total_registros|   media_interaction|
+--------+-----------------+---------------+--------------------+
| animals|       1593623628|             38|  9.55066085263158E7|
|   music|        922551152|             46|2.9691370304347824E7|
|     bed|        532691631|             44|       5.438209175E7|
| history|        440187490|             49| 1.565269257142857E7|
|   apple|        429916936|             42|1.0873628214285715E7|
| mrbeast|        300397699|             50|       6.896586282E7|
|  google|        239385460|             45|-3.35102360888888...|
|business|        210025196|             48|   7310180.020833333|
|   cubes|        170925917|             49|1.5043961224489795E7|
|  sports|        106924567|             49|   8695551.632653061|
| mukbang|         87433858|             45|1.1053630377777778E7|
|    lofi|         86445177|             40|         4167085.875|
|tutorial|

In [32]:
# calculando media e varianca de views por keyword
media_variancia_view = df_video.groupBy('Keyword').agg(
    avg('Views').alias('Media_views'),
    variance('views').alias('Variancia_views')
)
# mostrando media dos resultados
media_variancia_view.orderBy('Media_views', ascending=False).show(5)

+-------+--------------------+--------------------+
|Keyword|         Media_views|     Variancia_views|
+-------+--------------------+--------------------+
|animals| 9.472396092105263E7|8.353786825747299...|
|mrbeast|       6.676400398E7|3.824123679605851...|
| google| 6.143966791111111E7|1.026979729881539...|
|    bed| 5.389322861363637E7|1.166104831854568...|
|  music|2.9364893260869566E7|1.924797107187940...|
+-------+--------------------+--------------------+
only showing top 5 rows


In [31]:
# calcular media, minimo e maximo de views por keyword
estatisticas_view = df_video.groupBy('Keyword').agg(
    round(avg('views'), 0).alias('media_views'),
    min('views').alias('min_views'),
    max('views').alias('max_views')
)

# mostrando a media dos resultados
estatisticas_view.orderBy('media_views', ascending=False).show(5)

+-------+-----------+---------+----------+
|Keyword|media_views|min_views| max_views|
+-------+-----------+---------+----------+
|animals|9.4723961E7|    23448|1582262997|
|mrbeast|6.6764004E7|   889300| 285526909|
| google|6.1439668E7|     8064|2147483647|
|    bed|5.3893229E7|     4454| 524709805|
|  music|2.9364893E7|     2944| 915457091|
+-------+-----------+---------+----------+
only showing top 5 rows


In [43]:
# calcular o primeiro e o ultimo published at
published_dates = df_video.groupBy('Keyword').agg(
    min('Published At').alias('Primeiro_Published_At'),
    max('published At').alias('Ultimo_Published_At')
)

# mostrando resultados
published_dates.orderBy('Primeiro_published_At').show(5)

+---------+---------------------+-------------------+
|  Keyword|Primeiro_Published_At|Ultimo_Published_At|
+---------+---------------------+-------------------+
|      bed|           2007-07-16|         2022-08-24|
|education|           2008-07-25|         2022-08-24|
|  biology|           2009-02-16|         2022-07-30|
|    cubes|           2009-02-24|         2022-08-24|
|  animals|           2009-07-03|         2022-08-24|
+---------+---------------------+-------------------+
only showing top 5 rows


In [62]:
from pyspark.sql.functions import count, countDistinct

# contando todos os titles
contagem_titles = df_video.agg(
    count('Title').alias('total_titles'),
    countDistinct('Title').alias('total_titles_unicos')
)
#Resultado
contagem_titles.show()

+------------+-------------------+
|total_titles|total_titles_unicos|
+------------+-------------------+
|        1869|               1854|
+------------+-------------------+



In [52]:
# coletando tudo pra verificar se ha diferencas

resultado = contagem_titles.collect()[0]
total_titles = resultado['total_titles']
total_unicos = resultado['total_titles_unicos']

print(f'Total de titles: {total_titles}')
print(f'Total de titles unicos: {total_unicos}')
print(f'Diferencas: {total_titles - total_unicos}')
print(f'Ha títulos duplicados? {total_titles > total_unicos}')

if total_titles > total_unicos:
    print(f'Existem{total_titles - total_unicos} títulos duplicados')
else:
    print('todos os títulos sao unicos')

Total de titles: 1869
Total de titles unicos: 1854
Diferencas: 15
Ha títulos duplicados? True
Existem15 títulos duplicados


In [53]:
# contar registros por ano e ordenar em ordem ascendente
contagem_por_ano = df_video.groupBy('Year').agg(
    count('*').alias('quantidade_registros')
).orderBy('Year')

#Resultados
contagem_por_ano.show()

+----+--------------------+
|Year|quantidade_registros|
+----+--------------------+
|2007|                   2|
|2008|                   1|
|2009|                   9|
|2010|                   6|
|2011|                   4|
|2012|                  12|
|2013|                   6|
|2014|                  10|
|2015|                  15|
|2016|                  34|
|2017|                  47|
|2018|                  57|
|2019|                  86|
|2020|                 158|
|2021|                 229|
|2022|                1193|
+----+--------------------+



In [54]:
# contar registros por ano e mes, ordenando em ordem ascendente
contagem_ano_mes = df_video.groupBy('Year', 'Month').agg(
    count('*').alias('Quantidade_registros')
).orderBy('Year', 'Month')

#resultado
contagem_ano_mes.show()

+----+-----+--------------------+
|Year|Month|Quantidade_registros|
+----+-----+--------------------+
|2007|    7|                   1|
|2007|   12|                   1|
|2008|    7|                   1|
|2009|    2|                   2|
|2009|    6|                   2|
|2009|    7|                   1|
|2009|    8|                   1|
|2009|   10|                   1|
|2009|   12|                   2|
|2010|    3|                   1|
|2010|    5|                   2|
|2010|    6|                   1|
|2010|    9|                   1|
|2010|   10|                   1|
|2011|    2|                   1|
|2011|    5|                   1|
|2011|    9|                   1|
|2011|   10|                   1|
|2012|    1|                   1|
|2012|    2|                   3|
+----+-----+--------------------+
only showing top 20 rows


In [68]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col

# Definir a janela para cálculo acumulativo
window_spec = Window.partitionBy('Keyword').orderBy('Year')

# Calcular estatísticas anuais
estatisticas_anuais = df_video.groupBy("Keyword", "Year").agg(
    sum("Likes").alias("Soma_Anual"),
    count("Likes").alias("Contagem_Anual")
).orderBy("Keyword", "Year")

# Calcular médias acumulativas
media_acumulativa = estatisticas_anuais.withColumn(
    "Soma_Acumulativa",
    sum("Soma_Anual").over(window_spec)
).withColumn(
    "Contagem_Acumulativa",
    sum("Contagem_Anual").over(window_spec)
).withColumn(
    "Media_Acumulativa_Likes",
    col("Soma_Acumulativa") / col("Contagem_Acumulativa")
)

# resultado
media_acumulativa.select("Keyword", "Year", "Media_Acumulativa_Likes").orderBy("Keyword", "Year").show()

+-------+----+-----------------------+
|Keyword|Year|Media_Acumulativa_Likes|
+-------+----+-----------------------+
|animals|2009|              1357197.0|
|animals|2010|               587977.0|
|animals|2013|             3197276.75|
|animals|2014|     3258727.8333333335|
|animals|2019|     2950868.5714285714|
|animals|2020|           1723934.3125|
|animals|2021|            1186866.125|
|animals|2022|      760775.7894736842|
|  apple|2016|              4144389.0|
|  apple|2021|              2091325.0|
|  apple|2022|     118078.90476190476|
|   asmr|2020|               148120.0|
|   asmr|2021|              309373.25|
|   asmr|2022|      37351.06122448979|
|    bed|2007|               317160.5|
|    bed|2009|               659593.6|
|    bed|2010|               573747.5|
|    bed|2011|             621914.125|
|    bed|2017|     519308.07692307694|
|    bed|2018|      508940.5882352941|
+-------+----+-----------------------+
only showing top 20 rows
